In [ ]:
# ────────────────────────────────────────────────
# MILESTONE 3 – PCB Defect Detection App
# ────────────────────────────────────────────────

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import streamlit ultralytics pyngrok
!pip install -q streamlit ultralytics pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 90.4 MB/s eta 0:00:00


In [ ]:
# ─────────────────────────────────
#  PCB Defect Detection App
# ─────────────────────────────────

# Write the Streamlit application
%%writefile app.py
import streamlit as st
from ultralytics import YOLO
from PIL import Image, ImageDraw, ImageFont
import io
import torch
import os

# CONFIG
BASE_DIR  = "/content/drive/MyDrive/PCB_DATASET/"
MODEL_PATH = os.path.join(BASE_DIR, "best_yolov8_pcb.pt")

class_names = [
    'missing_hole', 'mouse_bite', 'open_circuit',
    'short', 'spur', 'spurious_copper'
]

# Load model (cached)
@st.cache_resource
def load_model():
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    st.write(f"**Model device:** {device.upper()}")
    return YOLO(MODEL_PATH)

model = load_model()

def run_inference_and_annotate(pil_image):
    results = model(pil_image, conf=0.25, iou=0.45)

    draw = ImageDraw.Draw(pil_image)
    try:
        font = ImageFont.truetype("arialbd.ttf", 22)
    except:
        try:
            font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf", 22)
        except:
            font = ImageFont.load_default()

    detections = []
    for box in results[0].boxes:
        cls_id = int(box.cls)
        conf  = float(box.conf)
        xyxy  = box.xyxy[0].cpu().numpy().tolist()
        label = class_names[cls_id]

        x1, y1, x2, y2 = map(int, xyxy)
        draw.rectangle((x1,y1,x2,y2), outline=(255,0,0), width=6)

        text = f"{label} {conf:.2f}"
        tx, ty = x1, y1 - 32

        # Text outline (black border)
        for dx, dy in [(-3,-3),(3,-3),(-3,3),(3,3),(-3,0),(3,0),(0,-3),(0,3)]:
            draw.text((tx+dx, ty+dy), text, fill=(0,0,0), font=font)
        draw.text((tx, ty), text, fill=(255,255,255), font=font)

        detections.append({"Defect": label, "Confidence": f"{conf:.3f}"})

    return pil_image, detections

#  STREAMLIT UI
st.set_page_config(page_title="PCB Defect Detector", layout="wide")

st.title("🔴 PCB Defect Detection System")
st.markdown("Upload a PCB image → defects are shown with **thick red boxes + labels**")

left, right = st.columns([1, 1.3])

with left:
    st.subheader("Upload Image")
    uploaded_file = st.file_uploader(
        "Choose a .jpg / .jpeg / .png file",
        type=["jpg", "jpeg", "png"]
    )

    if uploaded_file is not None:
        image = Image.open(uploaded_file).convert("RGB")
        st.image(image, caption="Uploaded Image", use_column_width=True)

with right:
    st.subheader("Detection Result")

    if uploaded_file is None:
        st.info("Upload an image on the left side")
    else:
        with st.spinner("Detecting defects ..."):
            annotated, det_list = run_inference_and_annotate(image.copy())

            if det_list:
                st.success(f"Found **{len(det_list)}** defect(s)")
                st.dataframe(det_list, use_container_width=True)
            else:
                st.warning("No defects detected")

            st.image(annotated, caption="Result", use_column_width=True)

            # Download
            buf = io.BytesIO()
            annotated.save(buf, format="JPEG")
            st.download_button(
                "📥 Download result",
                buf.getvalue(),
                file_name=f"detected_{uploaded_file.name}",
                mime="image/jpeg"
            )

st.markdown("---")
st.caption("YOLOv8 • 6 defect classes • Milestone 3")

Overwriting app.py


In [ ]:
# Download & prepare cloudflared (Cloudflare Tunnel)
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
!chmod +x cloudflared

In [ ]:
# Killing previous processes
!pkill -f streamlit    2>/dev/null || true
!pkill -f cloudflared  2>/dev/null || true

# Start Streamlit in background
!nohup streamlit run app.py --server.port 8501 --server.headless true > streamlit.log 2>&1 &

# Wait a moment
!sleep 8

# Start tunnel
!nohup ./cloudflared tunnel --url http://localhost:8501 > cf.log 2>&1 &

!sleep 10

# Show the public URL
!grep -o 'https://.*\.trycloudflare\.com' cf.log || echo "URL not ready yet — wait 5–15 seconds and run again"

^C
^C
https://hotels-projects-composition-food.trycloudflare.com
